# Overview
This notebook introduces you MONAI's image transformation module.

In [ ]:
# Copyright 2020 MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import sys
import numpy as np
import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

import monai
from monai.transforms import \
    LoadNifti, LoadNiftid, AddChanneld, ScaleIntensityRanged, \
    Rand3DElasticd, RandAffined, \
    Spacingd, Orientationd

monai.config.print_config()

## Data sources
Starting from a list of filenames. The Spleen dataset can be downloaded from http://medicaldecathlon.com/.

The following is a simple python script
to group pairs of image and label from `Task09_Spleen/imagesTr` and `Task09_Spleen/labelsTr`
folder. 

In [ ]:
data_root = '/workspace/data/medical/Task09_Spleen'

import os
import glob
train_images = sorted(glob.glob(os.path.join(data_root, 'imagesTr', '*.nii.gz')))
train_labels = sorted(glob.glob(os.path.join(data_root, 'labelsTr', '*.nii.gz')))
data_dicts = [{'image': image_name, 'label': label_name}
              for image_name, label_name in zip(train_images, train_labels)]
train_data_dicts, val_data_dicts = data_dicts[:-9], data_dicts[-9:]

The image file names are organised into a list of dictionaries.

In [ ]:
train_data_dicts[0]

The list of data dictionaries, `train_data_dicts`, could be used by
PyTorch's data loader.

For example,

```python
from torch.utils.data import DataLoader

data_loader = DataLoader(train_data_dicts)
for training_sample in data_loader:
    # run the deep learning training with training_sample
```

The rest of this tutorial presents a set of "transforms"
converting `train_data_dict` into data arrays that
will eventually be consumed by the deep learning models.

## Load the NIfTI files

One design choice of MONAI is that it provides not only the high-level workflow components,
but also relatively lower level APIs in their minimal functioning form.

For example, a `LoadNifti` class is a simple callable wrapper of the underlying `Nibabel` image loader.
After constructing the loader with a few necessary system parameters,
calling the loader instance with a NIfTI filename will return the image data arrays, as well as the metadata -- such as affine information and voxel sizes.

In [ ]:
loader = LoadNifti(dtype=np.float32)

In [ ]:
image, metadata = loader(train_data_dicts[0]['image'])
print(f"input: {train_data_dicts[0]['image']}")
print(f"image shape: {image.shape}")
print(f"image affine:\n{metadata['affine']}")
print(f"image pixdim:\n{metadata['pixdim']}")

Oftentimes, we want to load a group of inputs as a training sample.
For example training a supervised image segmentation network requires a pair of image and label as a training sample.

To ensure a group of inputs are beining preprocessed consistently,
MONAI also provides dictionary-based interfaces for the minimal functioning transforms.

`LoadNiftid` is the corresponding dict-based version of `LoadNifti`:

In [ ]:
loader = LoadNiftid(keys=('image', 'label'))

In [ ]:
data_dict = loader(train_data_dicts[0])
print(f"input:, {train_data_dicts[0]}")
print(f"image shape: {data_dict['image'].shape}")
print(f"label shape: {data_dict['label'].shape}")
print(f"image pixdim:\n{data_dict['image_meta_dict']['pixdim']}")

In [ ]:
image, label = data_dict['image'], data_dict['label']
plt.figure('visualize', (8, 4))
plt.subplot(1, 2, 1)
plt.title('image')
plt.imshow(image[:, :, 30], cmap='gray')
plt.subplot(1, 2, 2)
plt.title('label')
plt.imshow(label[:, :, 30])
plt.show()

## Add the channel dimension

Most of MONAI's image transformations assume that the input data has the shape:

`[num_channels, spatial_dim_1, spatial_dim_2, ... ,spatial_dim_n]`

so that they could be interpreted consistently (as "channel-first" is commonly used in PyTorch).

Here the input image has shape `(512, 512, 55)` which isn't in the acceptable shape (missing the channel dimension),

we therefore create a transform which is called to update the shape:

In [ ]:
add_channel = AddChanneld(keys=['image', 'label'])
datac_dict = add_channel(data_dict)
print(f"image shape: {datac_dict['image'].shape}")

Now we are ready to do some intensity and spatial transforms.

## Resample to a consistent voxel size

The input volumes might have different voxel sizes.

The following transform is created to normalise the volumes to have (1.5, 1.5, 5.) millimetre voxel size.

The transform is set to read the original voxel size information from `data_dict['image.affine']`,
which is from the corresponding NIfTI file, loaded earlier by `LoadNiftid`.

In [ ]:
spacing = Spacingd(keys=['image', 'label'], pixdim=(1.5, 1.5, 5.), mode=('bilinear', 'nearest'))

In [ ]:
data_dict = spacing(datac_dict)
print(f"image shape: {data_dict['image'].shape}")
print(f"label shape: {data_dict['label'].shape}")
print(f"image affine after Spacing:\n{data_dict['image_meta_dict']['affine']}") 
print(f"label affine after Spacing:\n{data_dict['label_meta_dict']['affine']}")

To track the spacing changes, the data_dict was updated by `Spacingd`:

- An `image.original_affine` key is added to the `data_dict`, logs the original affine.

- An `image.affine` key is updated to have the current affine.

In [ ]:
image, label = data_dict['image'], data_dict['label']
plt.figure('visualise', (8, 4))
plt.subplot(1, 2, 1)
plt.title('image')
plt.imshow(image[0, :, :, 30], cmap='gray')
plt.subplot(1, 2, 2)
plt.title('label')
plt.imshow(label[0, :, :, 30])
plt.show()

## Reorientation to a designated axes codes


Sometimes it is nice to have all the input volumes in a consistent axes orientation.

The default axis labels are Left (L), Right (R), Posterior (P), Anterior (A), Inferior (I), Superior (S).

The following transform is created to reorientate the volumes to have 'Posterior, Left, Inferior' (PLI) orientation:

In [ ]:
spacing = Orientationd(keys=['image', 'label'], axcodes='PLI')

In [ ]:
data_dict = spacing(data_dict)
print(f"image shape: {data_dict['image'].shape}")
print(f"label shape: {data_dict['label'].shape}")
print(f"image affine after Spacing:\n{data_dict['image_meta_dict']['affine']}") 
print(f"label affine after Spacing:\n{data_dict['label_meta_dict']['affine']}")

In [ ]:
image, label = data_dict['image'], data_dict['label']
plt.figure('visualise', (8, 4))
plt.subplot(1, 2, 1)
plt.title('image')
plt.imshow(image[0, :, :, 30], cmap='gray')
plt.subplot(1, 2, 2)
plt.title('label')
plt.imshow(label[0, :, :, 30])
plt.show()

## Random affine transformation

The following affine transformation is defined to output a (300, 300, 50) image patch.

The patch location is randomly chosen in a range of (-40, 40), (-40, 40), (-2, 2) in x, y, and z axes respectively.
The translation is relative to the image centre.

The 3D rotation angle is randomly chosen from (-45, 45) degrees around the z axis, and 5 degrees around x and y axes.

The random scaling factor is randomly chosen from (1.0 - 0.15, 1.0 + 0.15) along each axis.

In [ ]:
rand_affine = RandAffined(keys=['image', 'label'], mode=('bilinear', 'nearest'), prob=1.0,
                          spatial_size=(300, 300, 50),
                          translate_range=(40, 40, 2),
                          rotate_range=(np.pi/36, np.pi/36, np.pi*4),
                          scale_range=(0.15, 0.15, 0.15),
                          padding_mode='border')

You can rerun this cell to generate a different randomised version of the original image.

In [ ]:
affined_data_dict = rand_affine(data_dict)
print(f"image shape: {affined_data_dict['image'].shape}")

image, label = affined_data_dict['image'][0], affined_data_dict['label'][0]
plt.figure('visualise', (12, 6))
plt.subplot(1, 2, 1)
plt.title('image')
plt.imshow(image[:, :, 15], cmap='gray')
plt.subplot(1, 2, 2)
plt.title('label')
plt.imshow(label[:, :, 15])
plt.show()

## Random elastic deformation

Similarly, the following elastic deformation is defined to output a (300, 300, 10) image patch.

The image is resampled from a combination of affine transformations and elastic deformations.

`sigma_range` controls the smoothness of the deformation (larger than 15 could be slow on CPU)

`magnitude_range` controls the amplitude of the deformation (large than 500, the image becomes unrealistic).

In [ ]:
rand_elastic = Rand3DElasticd(
    keys=['image', 'label'], mode=('bilinear', 'nearest'), prob=1.0,
    sigma_range=(5, 8),
    magnitude_range=(100, 200),
    spatial_size=(300, 300, 10),
    translate_range=(50, 50, 2),
    rotate_range=(np.pi/36, np.pi/36, np.pi*2),
    scale_range=(0.15, 0.15, 0.15),
    padding_mode='border')

You can rerun this cell to generate a different randomised version of the original image.

In [ ]:
deformed_data_dict = rand_elastic(data_dict)
print(f"image shape: {deformed_data_dict['image'].shape}")

image, label = deformed_data_dict['image'][0], deformed_data_dict['label'][0]
plt.figure('visualise', (12, 6))
plt.subplot(1, 2, 1)
plt.title('image')
plt.imshow(image[:, :, 5], cmap='gray')
plt.subplot(1, 2, 2)
plt.title('label')
plt.imshow(label[:, :, 5])
plt.show()